In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os

# Replace with your actual directory
zip_file_path = '/content/drive/MyDrive/smartdiagnosisofclothflaw_round1train2_datasets.zip'
import zipfile

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/smartdiagnosisofclothflaw')

# Set the base directory to the unzipped content
base_dir = '/content/smartdiagnosisofclothflaw/guangdong1_round1_train2_20190828'
# List files in the extracted directory
os.listdir(base_dir)
# Set paths for normal and defect images, and annotations
normal_dir = os.path.join(base_dir, 'normal_Images')
defect_dir = os.path.join(base_dir, 'defect_Images')
annotations_path = os.path.join(base_dir, 'Annotations/anno_train.json')

# Continue with loading and processing the data as described before


Mounted at /content/drive


In [ ]:
!pip install tensorflow keras efficientnet
import tensorflow as tf
import pandas as pd
import efficientnet.tfkeras as efn
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import os
import json
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Set paths
base_dir = '/content/smartdiagnosisofclothflaw/guangdong1_round1_train2_20190828'
normal_dir = os.path.join(base_dir, 'normal_Images')
defect_dir = os.path.join(base_dir, 'defect_Images')
annotations_path = os.path.join(base_dir, 'Annotations/anno_train.json')

# Load annotations
with open(annotations_path) as f:
    annotations = json.load(f)

# Create lists for normal and defect images
normal_images = [os.path.join(normal_dir, img) for img in os.listdir(normal_dir)]
defect_images = [os.path.join(defect_dir, anno['name']) for anno in annotations]

# Create labels
normal_labels = [0] * len(normal_images)
defect_labels = [1] * len(defect_images)

# Combine normal and defect images and labels
images = np.array(normal_images + defect_images)
labels = np.array(normal_labels + defect_labels)

# Convert numeric labels to strings for binary classification
label_mapping = {0: 'normal', 1: 'defect'}
labels_str = np.array([label_mapping[label] for label in labels])

# Split into train and test sets
train_images, test_images, train_labels, test_labels = train_test_split(
    images, labels_str, test_size=0.2, random_state=42, stratify=labels_str
)

# Data generators with augmentation for the training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Generate data
train_generator = train_datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({'filename': train_images, 'class': train_labels}),
    x_col='filename',
    y_col='class',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({'filename': test_images, 'class': test_labels}),
    x_col='filename',
    y_col='class',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary',
    shuffle=False
)

# Build the model
def build_model():
    model = tf.keras.Sequential([
        efn.EfficientNetB0(input_shape=(224, 224, 3), include_top=False, weights='imagenet'),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

model = build_model()

# Callbacks for early stopping and model checkpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('efficientnet_best.keras', save_best_only=True, monitor='val_loss', mode='min')

# Training the model
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=10,
    callbacks=[early_stopping, model_checkpoint]
)

# Load the best model
model.load_weights('efficientnet_best.keras')

# Evaluate on test data
loss, accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

# Save the final model
model.save('efficientnet_fabric_defect.keras')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.1 MB/s eta 0:00:00
Found 2336 validated image filenames belonging to 2 classes.
Found 584 validated image filenames belonging to 2 classes.
16804768/16804768 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


73/73 ━━━━━━━━━━━━━━━━━━━━ 234s 2s/step - accuracy: 0.6194 - loss: 0.6541 - val_accuracy: 0.4075 - val_loss: 0.7706
Epoch 2/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 115s 1s/step - accuracy: 0.7323 - loss: 0.5205 - val_accuracy: 0.6370 - val_loss: 0.6884
Epoch 3/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 143s 1s/step - accuracy: 0.7787 - loss: 0.4628 - val_accuracy: 0.6216 - val_loss: 0.7071
Epoch 4/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 110s 1s/step - accuracy: 0.7803 - loss: 0.4466 - val_accuracy: 0.6935 - val_loss: 0.6403
Epoch 5/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 102s 1s/step - accuracy: 0.7854 - loss: 0.4353 - val_accuracy: 0.6695 - val_loss: 0.6610
Epoch 6/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 103s 1s/step - accuracy: 0.7935 - loss: 0.4080 - val_accuracy: 0.7551 - val_loss: 0.5606
Epoch 7/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 152s 1s/step - accuracy: 0.8025 - loss: 0.4270 - val_accuracy: 0.7175 - val_loss: 1.6611
Epoch 8/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 139s 1s/step - accuracy: 0.8230 - loss: 0.3878 - val_accuracy: 0.8202 - val_loss: 0.

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.activations import swish # import the swish activation function

# Load the fine-tuned model
model = tf.keras.models.load_model('/content/drive/MyDrive/efficientnet_fabric_defect_finetuned.keras', custom_objects={'swish': swish}) # add the custom object to the load_model function

# Directory of your test dataset
test_dir = '/content/fabric_test_data'

# Data generator for the test dataset (no augmentation, just rescaling)
test_datagen = ImageDataGenerator(rescale=1./255)

# Create a test data generator
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),  # Ensure this matches your model's input size
    batch_size=32,
    class_mode='binary',  # or 'categorical' for multi-class classification
    shuffle=False
)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

# Optionally, get predictions for each sample in the test set
predictions = model.predict(test_generator)

# If you want to map predictions back to labels
predicted_classes = [1 if pred > 0.5 else 0 for pred in predictions]  # For binary classification
# For multi-class classification:
# predicted_classes = predictions.argmax(axis=-1)

# Now you can compare these predictions to the true labels
true_classes = test_generator.classes


In [ ]:
!pip install tensorflow keras efficientnet
import tensorflow as tf
import pandas as pd
import efficientnet.tfkeras as efn
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import os
import json
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Set paths
base_dir = '/content/smartdiagnosisofclothflaw/guangdong1_round1_train2_20190828'
normal_dir = os.path.join(base_dir, 'normal_Images')
defect_dir = os.path.join(base_dir, 'defect_Images')
annotations_path = os.path.join(base_dir, 'Annotations/anno_train.json')

# Load annotations
with open(annotations_path) as f:
    annotations = json.load(f)

# Create lists for normal and defect images
normal_images = [os.path.join(normal_dir, img) for img in os.listdir(normal_dir)]
defect_images = [os.path.join(defect_dir, anno['name']) for anno in annotations]

# Create labels
normal_labels = [0] * len(normal_images)
defect_labels = [1] * len(defect_images)

# Combine normal and defect images and labels
images = np.array(normal_images + defect_images)
labels = np.array(normal_labels + defect_labels)

# Convert numeric labels to strings for binary classification
label_mapping = {0: 'normal', 1: 'defect'}
labels_str = np.array([label_mapping[label] for label in labels])

# Split into train and test sets
train_images, test_images, train_labels, test_labels = train_test_split(
    images, labels_str, test_size=0.2, random_state=42, stratify=labels_str
)

# Data generators with augmentation for the training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Generate data
train_generator = train_datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({'filename': train_images, 'class': train_labels}),
    x_col='filename',
    y_col='class',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({'filename': test_images, 'class': test_labels}),
    x_col='filename',
    y_col='class',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary',
    shuffle=False
)

# Build the model
def build_model():
    model = tf.keras.Sequential([
        efn.EfficientNetB0(input_shape=(224, 224, 3), include_top=False, weights='imagenet'),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

model = build_model()

# Callbacks for early stopping and model checkpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('efficientnet_best.keras', save_best_only=True, monitor='val_loss', mode='min')

# Training the model
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=10,
    callbacks=[early_stopping, model_checkpoint]
)

# Load the best model
model.load_weights('efficientnet_best.keras')

# Evaluate on test data
loss, accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

# Generate predictions
test_generator.reset()  # Reset the generator
predictions = model.predict(test_generator)
predicted_classes = (predictions > 0.5).astype(int)  # Convert probabilities to binary predictions

# Get true labels from the test generator
true_classes = test_generator.classes  # Actual labels
class_labels = list(test_generator.class_indices.keys())  # Class labels

# Generate classification report
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)

# Generate confusion matrix
confusion = confusion_matrix(true_classes, predicted_classes)
print("Confusion Matrix:")
print(confusion)

# Save the final model
model.save('efficientnet_fabric_defect.keras')


Found 2336 validated image filenames belonging to 2 classes.
Found 584 validated image filenames belonging to 2 classes.
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


73/73 ━━━━━━━━━━━━━━━━━━━━ 191s 2s/step - accuracy: 0.6423 - loss: 0.6258 - val_accuracy: 0.3887 - val_loss: 1.1146
Epoch 2/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 150s 1s/step - accuracy: 0.7496 - loss: 0.5112 - val_accuracy: 0.3955 - val_loss: 1.1895
Epoch 3/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 106s 1s/step - accuracy: 0.7638 - loss: 0.4704 - val_accuracy: 0.4760 - val_loss: 0.9703
Epoch 4/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 150s 1s/step - accuracy: 0.7903 - loss: 0.4401 - val_accuracy: 0.5771 - val_loss: 0.6617
Epoch 5/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 141s 1s/step - accuracy: 0.7780 - loss: 0.4570 - val_accuracy: 0.6130 - val_loss: 0.6142
Epoch 6/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - accuracy: 0.8290 - loss: 0.3857 - val_accuracy: 0.7757 - val_loss: 0.4632
Epoch 7/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 136s 1s/step - accuracy: 0.8098 - loss: 0.4088 - val_accuracy: 0.7158 - val_loss: 0.5637
Epoch 8/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 111s 1s/step - accuracy: 0.8191 - loss: 0.3938 - val_accuracy: 0.7911 - val_loss: 0.